In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mglearn
from IPython.display import display

%matplotlib inline


In [42]:
def unique_values(dataset, feature):
    return dataset[feature].unique()

def one_hotter_encoding(dataset, features):
    encoded = dataset.copy()
    
    for feature in features:
        unique_vals = unique_values(encoded, feature)
        for value in unique_vals:
            encoded[f"{feature}_{value}"] = (encoded[feature] == value).astype(float)
    
    return encoded


    
    
    
    

In [47]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [49]:
pd.options.display.max_rows = 15

In [59]:
train_pretty = train.drop("Cabin", axis=1)

In [81]:
train_pretty[train_pretty.drop(["Age"],axis=1).isnull().any(axis=1)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,NaN
